# Exploitation Zone - Visualització

- exploitation zone de la visualització
- preparation pipeline per taula d'entrenament del model --> cada zipcode és un indiv
    - Shops: latitu, longitud
    - Income: mitjana income per zipcode

In [ ]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *

#!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
#Arxiu Parquet 
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")
income = spark.read.parquet("./datalake/income_data/2024-04-22_IRSIncomeByZipCode_NoStateTotalsNoSmallZips.parquet")
sales = spark.read.parquet("./datalake/sales_data/2024-04-22_SuperstoreSalesTraining.parquet")

In [ ]:
# finalitzar sessió de Spark
spark.stop()